# UrbanSound8K Dataset (Taken from Kaggle)

## Import Libraries and Load Data

In [ ]:
import pandas as pd
import os
import csv
import librosa
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv("archive/UrbanSound8K.csv")
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


Exception ignored in: <function ResourceTracker.__del__ at 0x106e69bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x106ad9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


## Loading the Audio Files
With the load function of Librosa we are loading all of the audio files in our dataset and we store the audio files and labels in lists. so we can use them later for feature extracting and modeling.

In [ ]:
audio_files = []
labels = []

label_dict = {}
with open("archive/UrbanSound8K.csv") as f:
    reader = csv.reader(f)
    next(reader) 
    for row in reader:
        filename = row[0]
        label = row[7] 
        label_dict[filename] = label

for foldername in os.listdir("archive"):
    if foldername.endswith(".csv"): 
        continue
    for filename in os.listdir("archive/{}".format(foldername)):
        data, sr = librosa.load("archive/{}/{}".format(foldername, filename), sr=None)
        
        label = label_dict[filename]
        
        audio_files.append((data, sr))
        labels.append(label)

## Feature Extracting - MFCCs
**Mel-frequency cepstral coefficients (MFCCs)** are a set of features that capture the spectral characteristics of an audio signal. They are commonly used in speech and music processing. MFCCs are typically used to represent the spectral envelope of an audio signal, which is a summary of the spectral content of the signal over time. They are useful for tasks such as speaker recognition, speech recognition, and music classification, as they capture the spectral characteristics of the signal that are relevant to these tasks.

In [ ]:
X = []
max_length = 0
for data, sr in audio_files:
    mfccs = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
    X.append(mfccs_scaled)

/opt/anaconda3/lib/python3.13/site-packages/librosa/feature/spectral.py:2148: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


## Getting X and y ready
After Scaling the mfccs and using the label encoder on our labels, we can split our data into train and test datas.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
scaler = StandardScaler()
le = LabelEncoder()

X = scaler.fit_transform(X)
y = labels
y = le.fit_transform(labels)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modeling

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    SGDClassifier(),
    MLPClassifier(),
    XGBClassifier()
]

results_list = []


for clf in classifiers:
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    
    current_result = {
        'Classifier': clf.__class__.__name__,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall
    }
    
    results_list.append(current_result)

results = pd.DataFrame(results_list)

results.sort_values(by='Accuracy', ascending=False, inplace=True)
results

/opt/anaconda3/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,Classifier,Accuracy,Precision,Recall
2,RandomForestClassifier,0.906697,0.921385,0.893727
7,XGBClassifier,0.897539,0.908195,0.891478
0,KNeighborsClassifier,0.886090,0.888701,0.884878
6,MLPClassifier,0.884946,0.891681,0.883545
4,GradientBoostingClassifier,0.821408,0.827857,0.811719
1,DecisionTreeClassifier,0.693761,0.687378,0.686918
5,SGDClassifier,0.523755,0.521752,0.503568
3,AdaBoostClassifier,0.366915,0.434983,0.340383


Since the Random Forest model has the highest accuracy, we continue to finetune it using Optuna.

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion,
        random_state=42,
        n_jobs=-1
    )

    score = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=5).mean()
    
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("\nBest Hyperparameters:", study.best_params)
print("Best Accuracy:", study.best_value)

[I 2026-02-11 14:11:40,103] A new study created in memory with name: no-name-c8ab391e-f706-4b34-a4dd-47ee8dcc7ce4
[I 2026-02-11 14:11:45,102] Trial 0 finished with value: 0.8596993557623479 and parameters: {'n_estimators': 90, 'max_depth': 16, 'min_samples_split': 10, 'min_samples_leaf': 3, 'criterion': 'gini'}. Best is trial 0 with value: 0.8596993557623479.
[I 2026-02-11 14:11:47,348] Trial 1 finished with value: 0.8534001431639228 and parameters: {'n_estimators': 50, 'max_depth': 37, 'min_samples_split': 14, 'min_samples_leaf': 3, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8596993557623479.
[I 2026-02-11 14:11:59,158] Trial 2 finished with value: 0.8209019327129564 and parameters: {'n_estimators': 384, 'max_depth': 16, 'min_samples_split': 12, 'min_samples_leaf': 10, 'criterion': 'entropy'}. Best is trial 0 with value: 0.8596993557623479.
[I 2026-02-11 14:12:06,784] Trial 3 finished with value: 0.808303507516106 and parameters: {'n_estimators': 419, 'max_depth': 11, 'min


Best Hyperparameters: {'n_estimators': 324, 'max_depth': 48, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
Best Accuracy: 0.9032211882605583


Exception ignored in: <function ResourceTracker.__del__ at 0x1030a9bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x108cd1bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x1023c9bc0>
Traceback (most recent call last

## Save the model to run locally on ESP32

In [ ]:
import joblib
from sklearn.ensemble import RandomForestClassifier

best_rf = RandomForestClassifier(**study.best_params, random_state=42)

print("Training the optimized model with best parameters...")
best_rf.fit(X_train, y_train)

joblib.dump(best_rf, 'rf_model_tuned.joblib', compress=9)
joblib.dump(scaler, 'audio_scaler.joblib')

print("\n" + "="*40)
print("SUCCESS: Optimized Random Forest Saved!")
print(f"Best Params Used: {study.best_params}")
print("="*40)

Training the optimized model with best parameters...

SUCCESS: Optimized Random Forest Saved!
Best Params Used: {'n_estimators': 324, 'max_depth': 48, 'min_samples_split': 2, 'min_samples_leaf': 1, 'criterion': 'entropy'}
